In [1]:
#support vector machine

In [2]:
import pandas as pd
import sklearn as sk
import numpy as np

In [130]:
tweets=pd.read_csv("keyword+texto.csv")

In [131]:
a_predecir=pd.read_csv("keyword+texto_test.csv")

In [132]:
target_tweets=tweets['target']

In [133]:
def a_string(texto):
    return str(texto)

In [134]:
tweets=tweets['text'].apply(a_string)

In [135]:
a_predecir=a_predecir['text']

In [136]:
a_predecir=a_predecir.apply(a_string)

In [137]:
from sklearn.model_selection import train_test_split

In [118]:
x_train,x_validacion,y_train,y_validacion=train_test_split(tweets,target_tweets,test_size=0.1,random_state=1000)

### Creación del modelo

In [23]:
from sklearn.svm import SVC

In [95]:
from sklearn.svm import LinearSVC

In [24]:
def generar_clasificador(tipo_de_modelo):
    if tipo_de_modelo==1:
        return SVC(kernel='poly',degree=8,gamma='auto')
    if tipo_de_modelo==2:
        return SVC(kernel='rbf',gamma='auto')
    if tipo_de_modelo==3:
        return SVC(kernel='sigmoid',gamma='auto')
    if tipo_de_modelo==4:
        return SVC(kernel='linear',gamma='auto')

### Formas de generar los embeddings(?) de cada texto


#### Primer forma: count_vectorizer 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vectorizer=CountVectorizer()
count_vectorizer.fit(tweets)

In [ ]:
x_train=count_vectorizer.transform(x_train)

In [ ]:
x_validacion=count_vectorizer.transform(x_validacion)

#### Segunda forma: TFIDF 

In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [139]:
vectorizador=TfidfVectorizer()

In [140]:
vectorizador.fit(tweets)

TfidfVectorizer()

In [141]:
tweets=vectorizador.transform(tweets)

In [142]:
x_train,x_validacion,y_train,y_validacion=train_test_split(tweets,target_tweets,test_size=0.1,random_state=1000)

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,  181,  199,  681,  809,  121,  181,  199,  681,
        159, 1298,  136,    7,    0])

#### Tercera forma: usar tokenizer

In [118]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [119]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [120]:
tokenizer=Tokenizer()
todos_los_textos=np.concatenate([tweets,a_predecir])
tokenizer.fit_on_texts(todos_los_textos)
longitud_maxima=max([len(s.split()) for s in todos_los_textos])
print("longitud maxima: " + str(longitud_maxima))
#la longitud maxima de los textos es de 24 vamos a usar ese valor para los embeddings generados
tweets=tokenizer.texts_to_sequences(tweets)

longitud maxima: 24


In [121]:
tweets=pad_sequences(tweets,longitud_maxima*2)

In [122]:
#se generaron embeddings para todos los tweets de dimension 24

In [41]:
#ahora quiero ver si puedo combinar los valores obtenidos con los otros features generados
#voy a tener 24 + resto de features usados

In [123]:
dataframe_original=pd.read_csv("train_set_featurizado.csv")

In [124]:
caracteres_usados=dataframe_original['caracteres_usados'].values
#caracteres usados va a estar en la columna 25 de la matriz

In [125]:
dataframe_original.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'keyword', 'location', 'text',
       'target', 'caracteres_usados', 'menciones_realizadas',
       'permite_location', 'use_keyword', 'cita_url', 'use_hashtag',
       'numero_de_palabras', 'palabras_unicas',
       'numero_palabras_desconocidas'],
      dtype='object')

In [126]:
palabras_unicas=dataframe_original['palabras_unicas']
#va a estar en la posición 26 de la matriz

In [127]:
numero_palabras_desconocidas=dataframe_original['numero_palabras_desconocidas']
#va a estar en la posición 27 de la matriz

In [128]:
tweets=np.c_[tweets,caracteres_usados]

In [129]:
tweets=np.c_[tweets,palabras_unicas]

In [130]:
tweets=np.c_[tweets,numero_palabras_desconocidas]

In [131]:
tweets.shape

(7613, 51)

### Ejecución del código

#### Predicción realizada con Linear SVC usando únicamente solo el texto pre procesado

In [103]:
from sklearn.metrics import classification_report


In [107]:
modelo=LinearSVC(random_state=42)

In [108]:
modelo.fit(x_train,y_train)

LinearSVC(random_state=42)

In [109]:
y_predict=modelo.predict(x_validacion)

In [110]:
print(classification_report(y_predict,y_validacion))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83       478
           1       0.69      0.81      0.75       284

    accuracy                           0.80       762
   macro avg       0.78      0.80      0.79       762
weighted avg       0.81      0.80      0.80       762



#### Tuning

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import reciprocal, uniform

In [112]:
parametros=[{'loss':['hinge','squared_hinge'], 'C':uniform(1,10)}]

In [113]:
scores = ['precision', 'recall']
clf=RandomizedSearchCV(LinearSVC(),parametros)

In [114]:
clf.fit(x_train,y_train)

/home/tomas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


RandomizedSearchCV(estimator=LinearSVC(),
                   param_distributions=[{'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7228005518>,
                                         'loss': ['hinge', 'squared_hinge']}])

In [115]:
print("Mejores parametros encontrados en el training set: ")
print(clf.best_params_)

Mejores parametros encontrados en el training set: 
{'C': 1.0119102865709297, 'loss': 'squared_hinge'}


#### Predicción realizada con los mejores hiperparámetros

In [120]:
modelo=LinearSVC(C=1.0119102865709297,loss='squared_hinge')

In [121]:
modelo.fit(x_train,y_train)

LinearSVC(C=1.0119102865709297)

In [122]:
prediccion=modelo.predict(a_predecir)

#### Nuevo modelo, support vector classifier usando RBF como kernel

In [67]:
from sklearn.svm import SVC

In [68]:
svm_clf=SVC(gamma='scale')

In [69]:
svm_clf.fit(x_train,y_train)

SVC()

In [70]:
y_prediccion=svm_clf.predict(x_validacion)

In [74]:
from sklearn.metrics import accuracy_score

accuracy_score(y_prediccion,y_validacion)

0.8018372703412073

#### Hacemos tuning 

In [76]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}

In [77]:
clf=RandomizedSearchCV(SVC(),param_distributions)

In [78]:
clf.fit(x_train,y_train)

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f72254fb2b0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f72254fb710>})

In [79]:
clf.best_params_

{'C': 7.674095717579778, 'gamma': 0.04392746178318824}

#### Habiendo hecho el tuning, generamos el clasificador y hacemos la predicción


In [84]:
clasificador=SVC(C=7.674095717579778,gamma=0.04392746178318824)

In [87]:
clasificador.fit(x_train,y_train)

SVC(C=7.674095717579778, gamma=0.04392746178318824)

In [89]:
prediccion=clasificador.predict(x_validacion)

In [90]:
accuracy_score(prediccion,y_validacion)

0.800524934383202

In [102]:
a_predecir=vectorizador.transform(a_predecir)

AttributeError: lower not found

In [103]:
a_predecir

<3263x14018 sparse matrix of type '<class 'numpy.float64'>'
	with 24610 stored elements in Compressed Sparse Row format>

In [104]:
resultado=clasificador.predict(a_predecir)

In [105]:
resultado

array([1, 1, 1, ..., 1, 1, 0])

In [123]:
sampleSubmission=pd.read_csv('sample_submission.csv')
output=pd.DataFrame({'id':sampleSubmission.id,'target':prediccion})
output.to_csv('submission.csv',index=False)

### Probamos SVC con kernel polinomial

In [16]:
from sklearn.svm import SVC

In [17]:
svm_clf=SVC(kernel='poly',gamma='scale')

In [18]:
svm_clf.fit(x_train,y_train)

SVC(kernel='poly')

In [19]:
y_prediccion=svm_clf.predict(x_validacion)

In [20]:
from sklearn.metrics import accuracy_score

accuracy_score(y_prediccion,y_validacion)

0.7217847769028871

### Probamos SVC con kernel sigmoide

In [21]:
from sklearn.svm import SVC

In [23]:
svm_clf=SVC(kernel='sigmoid',gamma='scale')

In [24]:
svm_clf.fit(x_train,y_train)

SVC(kernel='sigmoid')

In [25]:
y_prediccion=svm_clf.predict(x_validacion)

In [26]:
from sklearn.metrics import accuracy_score

accuracy_score(y_prediccion,y_validacion)

0.800524934383202

In [31]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 20)}

In [32]:
clf=RandomizedSearchCV(SVC(),param_distributions)

In [33]:
clf.fit(x_train,y_train)

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb22a29d68>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb22a29f98>})

In [34]:
clf.best_params_

{'C': 20.3316680389878, 'gamma': 0.020962375508358482}

#### Habiendo hecho el tuning, generamos el clasificador y hacemos la predicción


In [36]:
clasificador=SVC(kernel='sigmoid',C=20.3316680389878,gamma=0.020962375508358482)

In [37]:
clasificador.fit(x_train,y_train)

SVC(C=20.3316680389878, gamma=0.020962375508358482, kernel='sigmoid')

In [38]:
prediccion=clasificador.predict(x_validacion)

In [39]:
accuracy_score(prediccion,y_validacion)

0.8031496062992126

In [40]:
a_predecir=vectorizador.transform(a_predecir)

In [45]:
resultado=clasificador.predict(a_predecir)

In [47]:
resultado.shape

(3263,)

In [51]:
sampleSubmission=pd.read_csv('sample_submission.csv')
sampleSubmission.shape

(3263, 2)

In [52]:
sampleSubmission=pd.read_csv('sample_submission.csv')
output=pd.DataFrame({'id':sampleSubmission.id,'target':resultado})
output.to_csv('submission.csv',index=False)

#### Nuevo modelo, support vector classifier usando RBF como kernel

In [143]:
from sklearn.svm import SVC

In [144]:
svm_clf=SVC(gamma='scale')

In [145]:
svm_clf.fit(x_train,y_train)

SVC()

In [146]:
y_prediccion=svm_clf.predict(x_validacion)

In [147]:
from sklearn.metrics import accuracy_score

accuracy_score(y_prediccion,y_validacion)

0.7874015748031497

#### Hacemos tuning 

In [148]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 20)}

In [149]:
clf=RandomizedSearchCV(SVC(),param_distributions)

In [150]:
clf.fit(x_train,y_train)

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb20a02b38>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb229b58d0>})

In [151]:
clf.best_params_

{'C': 6.1723981125612895, 'gamma': 0.040403912353425324}

#### Habiendo hecho el tuning, generamos el clasificador y hacemos la predicción


In [152]:
clasificador=SVC(C=6.1723981125612895,gamma=0.040403912353425324)

In [153]:
clasificador.fit(x_train,y_train)

SVC(C=6.1723981125612895, gamma=0.040403912353425324)

In [154]:
prediccion=clasificador.predict(x_validacion)

In [155]:
accuracy_score(prediccion,y_validacion)

0.7926509186351706

In [156]:
a_predecir=vectorizador.transform(a_predecir)

In [157]:
a_predecir

<3263x14021 sparse matrix of type '<class 'numpy.float64'>'
	with 25497 stored elements in Compressed Sparse Row format>

In [72]:
resultado=clasificador.predict(a_predecir)

In [73]:
resultado

array([1, 1, 1, ..., 1, 1, 0])

In [158]:
sampleSubmission=pd.read_csv('sample_submission.csv')
output=pd.DataFrame({'id':sampleSubmission.id,'target':a_predecir})
output.to_csv('submission.csv',index=False)

In [124]:
tweets['keyword'].fillna(' ',inplace=True)

61

In [125]:
a_predecir['keyword'].fillna(' ',inplace=True)

In [126]:
tweets['text']=tweets['text'].map(str) + " " + tweets['keyword'].map(str)

In [127]:
a_predecir['text']=a_predecir['text'].map(str)+ " "+a_predecir['keyword'].map(str)

In [128]:
tweets.to_csv('keyword+texto.csv')

In [129]:
a_predecir.to_csv('keyword+texto_test.csv')